In [10]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import requests
import json
import time
from datetime import datetime, date
from typing import Dict, List, Optional

## Loading datasets and analyzing features

In [11]:
aqi_cbsa_2025_df = pd.read_csv("../../Air_Quality_Datasets/annual_aqi_by_cbsa_2025.csv")
aqi_county_2025_df = pd.read_csv("../../Air_Quality_Datasets/annual_aqi_by_county_2025.csv")
conc_monitor_2025_df = pd.read_csv("../../Air_Quality_Datasets/annual_conc_by_monitor_2025.csv")

In [12]:
# aqi_cbsa_2025_df
# aqi_county_2025_df

In [13]:
conc_monitor_2025_df.columns

Index(['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC',
       'Latitude', 'Longitude', 'Datum', 'Parameter Name', 'Sample Duration',
       'Pollutant Standard', 'Metric Used', 'Method Name', 'Year',
       'Units of Measure', 'Event Type', 'Observation Count',
       'Observation Percent', 'Completeness Indicator', 'Valid Day Count',
       'Required Day Count', 'Exceptional Data Count', 'Null Data Count',
       'Primary Exceedance Count', 'Secondary Exceedance Count',
       'Certification Indicator', 'Num Obs Below MDL', 'Arithmetic Mean',
       'Arithmetic Standard Dev', '1st Max Value', '1st Max DateTime',
       '2nd Max Value', '2nd Max DateTime', '3rd Max Value',
       '3rd Max DateTime', '4th Max Value', '4th Max DateTime',
       '1st Max Non Overlapping Value', '1st NO Max DateTime',
       '2nd Max Non Overlapping Value', '2nd NO Max DateTime',
       '99th Percentile', '98th Percentile', '95th Percentile',
       '90th Percentile', '75th Percentile', '

In [14]:
conc_monitor_2025_df

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,...,75th Percentile,50th Percentile,10th Percentile,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change
0,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,1 HOUR,...,0.054,0.045,0.029,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2025-10-29
1,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.049,0.040,0.026,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2025-10-29
2,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.049,0.040,0.026,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2025-10-29
3,1,3,10,44201,1,30.497478,-87.880258,NAD83,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.049,0.040,0.026,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2025-10-29
4,1,3,10,88101,3,30.497478,-87.880258,NAD83,PM2.5 - Local Conditions,1 HOUR,...,10.000,6.000,1.000,"FAIRHOPE, Alabama","FAIRHOPE HIGH SCHOOL, 1 PIRATE DRIVE, FAIRHOPE...",Alabama,Baldwin,Fairhope,"Daphne-Fairhope-Foley, AL",2025-10-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59149,80,26,6,88101,1,31.291293,-110.951513,WGS84,PM2.5 - Local Conditions,24-HR BLK AVG,...,12.200,9.300,5.900,Nogales Sonora Institute ITN,"Avenida Instituto Tecnologico #911, Granja, 84...",Country Of Mexico,SONORA,NaN,NaN,2025-10-22
59150,80,26,8012,44201,1,32.466389,-114.768611,WGS84,Ozone,1 HOUR,...,0.058,0.050,0.036,San Luis Rio Colorado Well 10,Calle 15 & Venustiano Carranza Water Well #10,Country Of Mexico,SONORA,NaN,NaN,2025-10-27
59151,80,26,8012,44201,1,32.466389,-114.768611,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.053,0.046,0.034,San Luis Rio Colorado Well 10,Calle 15 & Venustiano Carranza Water Well #10,Country Of Mexico,SONORA,NaN,NaN,2025-10-27
59152,80,26,8012,44201,1,32.466389,-114.768611,WGS84,Ozone,8-HR RUN AVG BEGIN HOUR,...,0.053,0.046,0.034,San Luis Rio Colorado Well 10,Calle 15 & Venustiano Carranza Water Well #10,Country Of Mexico,SONORA,NaN,NaN,2025-10-27


In [15]:
# Notes about features:
# Sample Duration is the length of time that air passes through the monitoring
# device before it is measured.
# Is there significance to this? Can we compare these measures for different but
# similar locations?
# Metric Use, Method Name, and Pollutant standard describe how the sample is 
# aggregated.

# Pollutant standardA description of the ambient air quality standard rules 
# used to aggregate statistics.

# Units of Measure (IMPORTANT): The unit for measuring the parameter
# How will we convert to a standard unit? What is the standard for this?

# Event Type: Tells us in an event such as a wildfire maybe have happened 
# and was included or not

# Observation Count: number of samples taken during the year
# Observation Percent: the number of samples taken versus number scheduled to be taken
# Completeness Indicator (IMPORTANT): Y if the regulatory data completeness criteria
# for a valid summary was met. N, if there is no criteria or if criteria was not met
# Valid day count is the number of days criteria was met
# Required Day count is the number of days required that year for the monitor to
# to take samples.

# Exceptional Data Count:


In [16]:
# How many state, county, sites do we have?
print(F"Number of unique state codes: {len(pd.unique(conc_monitor_2025_df["State Code"]))}")
print(F"Number of unique county codes: {len(pd.unique(conc_monitor_2025_df["County Code"]))}")
print(F"Number of unique site nums: {len(pd.unique(conc_monitor_2025_df["Site Num"]))}")

Number of unique state codes: 54
Number of unique county codes: 160
Number of unique site nums: 477


In [17]:
# Is site number always the same latitude and longitude?
site_locs = conc_monitor_2025_df.groupby("Site Num")[["Latitude", "Longitude"]].first()
print(F"Is len(site_locs) ({len(site_locs)}) = #of unique site nums ({len(pd.unique(conc_monitor_2025_df["Site Num"]))})")

Is len(site_locs) (477) = #of unique site nums (477)


In [18]:
# How many monitors do we have per site?
monitors_per_site = conc_monitor_2025_df.groupby(
    ["Site Num"]
)["Parameter Code"].nunique().reset_index()
monitors_per_site.columns = ["Site Num", "Num Monitors"]

# Summary statistics
print("Summary statistics of monitors per site combination:")
print(monitors_per_site["Num Monitors"].describe())

# So maybe we find the summary statistics of:
# #of monitors per particle over all sites
monitors_per_siteparticle = conc_monitor_2025_df.groupby(
    ["Site Num", "Parameter Name"]
)["Parameter Code"].nunique().reset_index()
monitors_per_siteparticle.columns = ["Site Num", "Parameter Name", "Num Monitors"]

Summary statistics of monitors per site combination:
count    477.000000
mean      39.436059
std       64.398433
min        1.000000
25%        4.000000
50%        9.000000
75%       46.000000
max      299.000000
Name: Num Monitors, dtype: float64


In [19]:
# List all the unique parameters
conc_monitor_2025_df["Parameter Code"].unique()

# How many different parameters does each site monitor have
# Create a unique site identifier (state+county+sitenumber)
conc_monitor_2025_df['Site_ID'] = (
    conc_monitor_2025_df['State Code'].astype(str) + '-' +
    conc_monitor_2025_df['County Code'].astype(str) + '-' +
    conc_monitor_2025_df['Site Num'].astype(str)
)

# Count unique parameter codes per site
params_per_site = conc_monitor_2025_df.groupby('Site_ID')['Parameter Code'].nunique().reset_index()
params_per_site.columns = ['Site_ID', 'Num_Parameters']

# Summary statistics
print("Summary statistics of parameters per site:")
print(params_per_site['Num_Parameters'].describe())
print()

Summary statistics of parameters per site:
count    2246.000000
mean       16.437222
std        32.059554
min         1.000000
25%         1.000000
50%         5.000000
75%        11.000000
max       266.000000
Name: Num_Parameters, dtype: float64



In [20]:
# Via the api extract the daily data for 2025 for all monitors

## DIFFERNT PRE-GENED FILE TYPES & DATA USED:  
### MONITOR DESCRIPTION FILE

A unique monitor is identified by the combination of:
- state code  
- county code  
- site number (within county)
- parameter code  
- parameter occurrence code ("POC", used to differentiate when a parameter is measured more than once at a site).


Daily Summary Data:  
*Returns data summarized at the daily level. All daily summaries are calculated on midnight to midnight basis in local time. Variables returned include date, mean value, maximum value, etc. Data is at the monitor level and may include more than one entry per monitor. There may be multiple entries for different (1) sample durations, (2) pollutant standards.*

In [21]:
# GLOBAL ENVIRONMENT VARS:
MY_EMAIL = "huex@rose-hulman.edu"
MY_KEY = "orangecat14"

In [22]:
# PARAMETER CODES AND DESCRIPTIONS (SIMPLIFY THE PROBLEM)
ALL_PARAMS = {
    "Carbon Monoxide": {
        "parameter_code": 42101,
        "abbreviation": "CO",
        "description": "Carbon monoxide concentration (ppm)"
    },
    "Nitrogen Dioxide": {
        "parameter_code": 42602,
        "abbreviation": "NO2",
        "description": "Nitrogen dioxide concentration (ppb)"
    },
    "Ozone": {
        "parameter_code": 44201,
        "abbreviation": "O3",
        "description": "Ozone concentration (ppm)"
    },
    "Sulfur Dioxide": {
        "parameter_code": 42401,
        "abbreviation": "SO2",
        "description": "Sulfur dioxide concentration (ppb)"
    },
    "PM10 (Coarse Particulate Matter)": {
        "parameter_code": 81102,
        "abbreviation": "PM10",
        "description": "Particulate matter with diameter ≤ 10 μm (µg/m³)"
    },
    "PM2.5 (Fine Particulate Matter)": {
        "parameter_code": 88101,
        "abbreviation": "PM2.5",
        "description": "Particulate matter with diameter ≤ 2.5 μm (µg/m³)"
    },
    "Lead": {
        "parameter_code": 14129,
        "abbreviation": "Pb (TSP) LC",
        "description": "Lead measured as total suspended particulates (µg/m³)"
    }
}

In [23]:
len(ALL_PARAMS)

7

In [ ]:
# API Retrival Functions
def fmt_scode(sc):
    return f"{sc:02d}"

def fmt_yyyymmdd(d):
    # Accepts str, date/datetime, or pandas Timestamp and returns 'YYYYMMDD'
    return pd.to_datetime(d).strftime('%Y%m%d')

def get_daily_data_per_year(all_data, start_year, params, state):
    url = "https://aqs.epa.gov/data/api/dailyData/byState"
    #all_data = []

    # get the end_date (one year from start)
    start_date = date(start_year, 1, 1)
    end_date   = date(start_year, 12, 31)

    # Convert dates to a single string
    start_date = fmt_yyyymmdd(start_date)
    end_date   = fmt_yyyymmdd(end_date)

    # Get all the parameter codes
    param_codes = [params["parameter_code"] for params in ALL_PARAMS.values()]

    for param_code in param_codes:
        time.sleep(5) # Wait 10 per each param, 7 requests per 70 seconds, limit is 10 per min
        state_code = fmt_scode(state)

        params = {
            'email': MY_EMAIL,
            'key': MY_KEY,
            'param': param_code,
            'bdate': start_date,
            'edate': end_date,
            'state': state_code
        }
        try:
            response = requests.get(url, params=params, timeout=60)  # Use GET, not POST
            data = response.json()
            if data:
                all_data.extend(data["Data"])
        except requests.exceptions.Timeout:
            print(f"Param {param_code}: Skipped (timeout after 60s)")
            continue
        except requests.exceptions.RequestException as e:
            print(f"✗ Param {param_code}: Error - {e}")
            continue
    return

In [25]:
import os

In [45]:
# Extracting daily data over a year using the API
csv_file = "air_quality_data.csv"

# Load existing data if file exists
if os.path.exists(csv_file):
    existing_df = pd.read_csv(csv_file)
    all_data = existing_df.to_dict('records')
else:
    all_data = []

# Fetch new data (manually change state code as needed)
for i in range(8, 55):
    get_daily_data_per_year(all_data, 2024, ALL_PARAMS, i)  # Change state code here

# Save combined data
test_df = pd.DataFrame(all_data)
test_df.to_csv(csv_file, index=False)

print(f"Total rows: {len(test_df)}")


Param 44201: Skipped (timeout after 60s)
Param 42401: Skipped (timeout after 60s)
Param 88101: Skipped (timeout after 60s)
Param 14129: Skipped (timeout after 60s)
Param 44201: Skipped (timeout after 60s)
Param 42401: Skipped (timeout after 60s)
Param 88101: Skipped (timeout after 60s)
Param 88101: Skipped (timeout after 60s)
Param 88101: Skipped (timeout after 60s)
Param 42401: Skipped (timeout after 60s)
Param 88101: Skipped (timeout after 60s)
Param 44201: Skipped (timeout after 60s)
Param 88101: Skipped (timeout after 60s)


: 

: 

In [44]:
test_df = pd.read_csv("air_quality_data.csv")
test_df

,state_code,county_code,site_number,parameter_code,poc,latitude,longitude,datum,parameter,sample_duration_code,...,method_code,method,local_site_name,site_address,state,county,city,cbsa_code,cbsa,date_of_last_change
0,1,73,23,42101,2,33.553056,-86.815000,WGS84,Carbon monoxide,1,...,93,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,13820.0,"Birmingham-Hoover, AL",2025-07-09
1,1,73,23,42101,2,33.553056,-86.815000,WGS84,Carbon monoxide,Z,...,93,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,13820.0,"Birmingham-Hoover, AL",2025-07-09
2,1,73,23,42101,2,33.553056,-86.815000,WGS84,Carbon monoxide,1,...,93,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,13820.0,"Birmingham-Hoover, AL",2025-07-09
3,1,73,23,42101,2,33.553056,-86.815000,WGS84,Carbon monoxide,Z,...,93,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,13820.0,"Birmingham-Hoover, AL",2025-07-09
4,1,73,23,42101,2,33.553056,-86.815000,WGS84,Carbon monoxide,1,...,93,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,13820.0,"Birmingham-Hoover, AL",2025-07-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452393,6,37,1405,14129,2,34.006389,-118.193056,WGS84,Lead (TSP) LC,7,...,261,HI-VOL - Microwave Assisted Digestion Quadrapo...,Rehrig (Exide),"4010 E. 26th Street, Vernon, CA 90058",California,Los Angeles,Vernon,31080.0,"Los Angeles-Long Beach-Anaheim, CA",2024-12-20
452394,6,37,1405,14129,2,34.006389,-118.193056,WGS84,Lead (TSP) LC,7,...,261,HI-VOL - Microwave Assisted Digestion Quadrapo...,Rehrig (Exide),"4010 E. 26th Street, Vernon, CA 90058",California,Los Angeles,Vernon,31080.0,"Los Angeles-Long Beach-Anaheim, CA",2024-12-20
452395,6,37,1405,14129,2,34.006389,-118.193056,WGS84,Lead (TSP) LC,7,...,261,HI-VOL - Microwave Assisted Digestion Quadrapo...,Rehrig (Exide),"4010 E. 26th Street, Vernon, CA 90058",California,Los Angeles,Vernon,31080.0,"Los Angeles-Long Beach-Anaheim, CA",2024-12-20
452396,6,37,1405,14129,2,34.006389,-118.193056,WGS84,Lead (TSP) LC,7,...,261,HI-VOL - Microwave Assisted Digestion Quadrapo...,Rehrig (Exide),"4010 E. 26th Street, Vernon, CA 90058",California,Los Angeles,Vernon,31080.0,"Los Angeles-Long Beach-Anaheim, CA",2024-12-20


## Some Questions:  
Does the monitor position have an affect on the way aqs is measured?  
Is there a way to see if we are collecting accurate data at each monitor?  

## Plotting geographical data  
Im bored of data exploring I want to see what a map will look like

In [17]:
# Create new frame with Hawaii and Alaska filtered out for better map view
cm_2025_nha_df = conc_monitor_2025_df[(conc_monitor_2025_df["State Name"] != "Alaska") & 
                                      (conc_monitor_2025_df["State Name"] != "Hawaii") &
                                      (conc_monitor_2025_df["State Name"] != "Puerto Rico") &
                                      (conc_monitor_2025_df["State Name"] != "Virgin Islands") &
                                      (conc_monitor_2025_df["State Name"] != "Country Of Mexico")]

In [20]:
cm_2025_nha_df['State Name'].unique()

array(['Alabama', 'Arizona', 'Arkansas', 'California', 'Colorado',
       'Connecticut', 'Delaware', 'District Of Columbia', 'Florida',
       'Georgia', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas',
       'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts',
       'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana',
       'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico',
       'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma',
       'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina',
       'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont',
       'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype=object)

In [21]:
# Convert your DataFrame to GeoDataFrame
geometry = [Point(xy) for xy in zip(cm_2025_nha_df['Longitude'], cm_2025_nha_df['Latitude'])]
gdf = gpd.GeoDataFrame(cm_2025_nha_df, geometry=geometry, crs='EPSG:4326')

In [23]:
# Download US states basemap (optional but recommended)
usa = gpd.read_file('https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_20m.zip')

# Create the plot
fig, ax = plt.subplots(figsize=(150, 100))

# Plot base map
usa.boundary.plot(ax=ax, linewidth=3, color='gray')

# Plot your monitoring sites
# Plot points colored by arithmetic mean
gdf.plot(
    ax=ax,
    column='Arithmetic Mean',
    marker='o',
    cmap='RdYlGn_r',  # Red-Yellow-Green reversed (red = high pollution)
    markersize=100,
    alpha=0.7,
    legend=True,
    legend_kwds={'label': f"Pollution Level ({cm_2025_nha_df['Units of Measure'].iloc[0]})", 
                 'orientation': 'vertical'}
)

# Auto-fit to your data bounds with padding
minx, miny, maxx, maxy = gdf.total_bounds
margin = 0.1  # 10% margin
width = maxx - minx
height = maxy - miny

ax.set_xlim(minx - margin * width, maxx + margin * width)
ax.set_ylim(miny - margin * height, maxy + margin * height)

plt.title('Air Quality - Pollution Levels by Site')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.tight_layout()
plt.show()